<a href="https://colab.research.google.com/github/lymoelopez/automated-filipino-fake-news-detector/blob/main/evidenceCollector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evidence Collector

## I. Web Searcher

#### Web Search Filter

In [1]:
URL_banList = ["facebook", "twitter", "youtube", "pdf", "blog", "tiktok", "instagram", "youtu.be", "mp4", "mp3", "audiobook", "podcast", "spotify", "slideshare", "github", "huggingface"]

def URLfilter(url):
  lowercasedURL = url.lower()
  if all(excludedURL not in lowercasedURL for excludedURL in URL_banList):
    return url

#### Google Search Api

In [2]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install google

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from googlesearch import search

def webSearcher(inputClaim, numOfResults=20):

  filteredList = []
  #numOfResults = 20
  
  while len(filteredList) <= numOfResults:
    if len(filteredList) >= numOfResults:
      break
    for url in search(inputClaim, lang="tl", num=numOfResults, pause=3):
      filteredURL = URLfilter(url)
      if filteredURL and (filteredURL not in filteredList):
        filteredList.append(filteredURL)
      if len(filteredList) >= numOfResults:
        break

    return filteredList

## II. Evidence Selector

#### Article Extractor

In [5]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 72.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=ab3b0d559ba850290fb68b0dc0eb67fed305fd61875b34ae502c485ea01d4b02
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3356 sha256=a53214ffa973a2ccbe36a8aba69de80810c3c3464a3463375b05

In [6]:
from newspaper import Article
from newspaper import Config

def articleExtractor(url, articleExtractorConfig):

  try:
    article = Article(url, config=articleExtractorConfig)
    article.download()
    article.parse()
  except:
    return ""

  return article.text

In [7]:
def createExtractedArticlesList(urlList):

  extractedArticlesList = []

  articleExtractorConfig = Config()
  articleExtractorConfig.requests_timeout = 30

  for url in urlList:
    extractedArticlesList.append(articleExtractor(url, articleExtractorConfig))

  return extractedArticlesList

#### Cosine Similarity

In [8]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=c5f591f7180502c937b9cbd795146cfc6884f84d318d40c937a98cf0c1242ca3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [9]:
from sentence_transformers import SentenceTransformer

cosineSimilarityModel = SentenceTransformer("danjohnvelasco/filipino-sentence-roberta-v1")

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def findSimilarityScores(inputClaim, extractedArticlesList, model):

  # append claim in front of extracted articles list
  sentences = list(extractedArticlesList)
  sentences.insert(0, inputClaim)
  sentenceEmbeddings = model.encode(sentences)

  similarityScores = cosine_similarity(
      [sentenceEmbeddings[0]],
      sentenceEmbeddings[1:]
  )

  return similarityScores[0]

In [14]:
import numpy as np

def findTopFiveEvidences(inputClaim, extractedArticlesList, model):

  similarityScores = findSimilarityScores(inputClaim, extractedArticlesList, model)
  topFiveSimilarityScoresIndex = np.argpartition(similarityScores,-5)[-5:]

  numpyExtractedArticlesList = np.array(extractedArticlesList)
  topFiveEvidences = numpyExtractedArticlesList[topFiveSimilarityScoresIndex]

  return topFiveEvidences

# Evidence Collector Function

In [12]:
def evidenceCollector(inputClaim, model):

  filteredUrlList = webSearcher(inputClaim, 20)
  extractedArticlesList = createExtractedArticlesList(filteredUrlList)
  topFiveEvidences = findTopFiveEvidences(inputClaim, extractedArticlesList, model)

  return filteredUrlList, extractedArticlesList, topFiveEvidences

### Sample Claims

#### Sample Claim 1

In [15]:
sampleClaim1 = "may banta sa buhay ni Teves"
filteredUrlList1, extractedArticlesList1, topFiveEvidences1 = evidenceCollector(sampleClaim1, cosineSimilarityModel)

In [16]:
len(filteredUrlList1)

20

In [17]:
len(extractedArticlesList1)

20

In [18]:
print(topFiveEvidences1)

['Nakiusap muli si Pangulong Ferdinand Marcos, Jr. kay Negros Oriental (3rd District) Rep. Arnolfo Teves, Jr. na umuwi na sa bansa upang harapin ang mga alegasyon laban sa kanya.\n\nPaniniyak ng Pangulo, ibibigay nila ang lahat ng klase ng seguridad sa pagbabalik ni Teves sa bansa.\n\n“But anyway, if to reassure him, we’ll provide all kinds of security kung anong gusto mo. Mayaman ka naman. May private jet ka naman eh. Mag-landing ka kung saan mo gusto, papaligiran – sa Air Force base. Mag-landing siya sa Basa, papaligiran natin ng sundalo, walang makalapit na isang kilometro. So that – that will guarantee his security,” reaksyon ni Marcos matapos dumalo sa ika-126 anibersaryo ng Philippine Army sa Taguig City nitong Miyerkules.\n\nItinanggi rin nito ang ulat sa sinasabing banta sa buhay ni Teves batay na rin nakalap na intelligence information ng Office of the President.\n\n“Well, I have no direct contact with him. Ang nakakausap niya is the Speaker (Martin Romualdez) dahil ‘yun ang S

#### Sample Claim 2

In [19]:
sampleClaim2 = "Wala nang cancer si Tenorio."
filteredUrlList2, extractedArticlesList2, topFiveEvidences2 = evidenceCollector(sampleClaim2, cosineSimilarityModel)

/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [20]:
len(filteredUrlList2)

20

In [21]:
len(extractedArticlesList2)

20

In [22]:
print(topFiveEvidences2)

['Ipinabatid ni Ginebra point guard LA Tenorio ang tunay na lagay ng kanyang kalusugan.\n\nSa ipinalabas na opisyal na pahayag, sinabi ni Tenorio na meron siyang sakit na kanser at stage 3 na ito.\n\n“I would like to issue a statement about my health status by firstly apologizing to my teammates, some coaches, the PBA, the fans, the media and even some friends. As most of you are aware I have been nursing a minor injury since the Finals last January. I used that as the reason for my sudden absence. My sincerest apologies to all,” bahagi ng statement ni Tenorio.\n\nAniya, minabuti niyang maglabas na lamang ng official statement upang maiwasan ang mga fake news at iba pang haka-haka.\n\n“I was recently diagnosed with Stage 3 colon cancer. The initial testing three weeks ago led me to instantly miss practices and games,” aniya.\n\nDagdag pa niya, sumailalim na siya sa kinakailangang operasyon, “I have completed my surgery last week and will soon undergo treatment for the next few months.”

#### Sample Claim 3

In [23]:
sampleClaim3 = "Ika-2 ang PH sa Asia-Pacific sa mga namamatay dahil sa polusyon sa bahay"
filteredUrlList3, extractedArticlesList3, topFiveEvidences3 = evidenceCollector(sampleClaim3, cosineSimilarityModel)

/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [26]:
len(filteredUrlList3)

18

In [27]:
len(extractedArticlesList3)

18

In [28]:
print(topFiveEvidences3)

[''
 'Pollution caused by traditional cooking fuel is killing people in the Philippines at higher rates than those in most other countries in the Asia-Pacific region, according to the World Health Organization (WHO).\n\nThe Philippines has earned the distinction of having the region’s second biggest number of deaths due to indoor air pollution, WHO said in a recent report.\n\nADVERTISEMENT\n\nData gathered by the world health agency showed that close to 84 deaths per 100,000 Filipinos were recorded in 2016 due to indoor or household air pollution.\n\nThis put the Philippines second only to Laos, where at least 85 deaths per 100,000 population were recorded.\n\nThe Philippines also ranked third in the number of deaths at 45.3 per 100,000 due to outdoor air pollution. China recorded 81.5 deaths, and Mongolia, 48.8.\n\n“Air pollution is the most lethal environmental health threat in our region, and it affects people in middle-income countries at a much higher rate than those in high-incom

# References

[1] https://www.geeksforgeeks.org/performing-google-search-using-python-code/

[2] https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1

[3] https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array